In [36]:
import copy
import pandas as pd 

## 首先对类别标签进行处理

如果一个xlsx中有多个sheet,那么在read_excel的时候把sheet_name参数设置一下就好了，

index_col是指定哪一列作为列索引的，我这里就用企业代号作为索引了。

encoding='gb18030'是控制编码格式，防止在有中文的时候出现乱码，不在读写和打开的时候发现有乱码。

In [37]:
data_1_1 = pd.read_excel('Data\附件1：123家有信贷记录企业的相关数据.xlsx',sheet_name='企业信息',index_col=0,encoding='gb18030')

简单地打开,处理一下看看

In [38]:
# 我没有打算用到企业名称，所以我想把它删掉,其实drop有有一个inplace参数，可以选择是否就在操作对象上进行数据的删除，后面再用
data_1_1_ed = data_1_1.drop(axis=1,columns='企业名称')
# 将"是"与"否"转化为1和0
data_1_1_ed.loc[data_1_1_ed['是否违约']=='是', '是否违约'] = 1
data_1_1_ed.loc[data_1_1_ed['是否违约']=='否', '是否违约'] = 0
# 打印出来看看
data_1_1_ed.head()

,信誉评级,是否违约
企业代号,,
E1,A,0
E2,A,0
E3,C,0
E4,C,0
E5,B,0


先复制一份处理好的数据，等一下就知道我为什么要这么做了

In [39]:
data_1_1_ed_2 = copy.copy(data_1_1_ed)

使用get_dummies函数,对信誉等级进行独热编码，注意索引的方式

.iloc是DataFrame的索引器，通过它，我们可以像对待Numpy数组一样索引Pandas的的底层数组，DataFrame的行列标签会自动保留在结果中

让我们打印出前五行看看

In [40]:
one_hoted_score = pd.get_dummies(data_1_1_ed[['信誉评级']])
one_hoted_score.iloc[:5,:]

,信誉评级_A,信誉评级_B,信誉评级_C,信誉评级_D
企业代号,,,,
E1,1,0,0,0
E2,1,0,0,0
E3,0,0,1,0
E4,0,0,1,0
E5,0,1,0,0


使用concat拼接，axis=1的话我也不知道怎么说，就是在列上进行拼接吧，不增加行数的那种，打出来看看就知道了

将inplace参数设置为True,表示就在这个Dataframe上进行修改

In [41]:
data_1_1_ed = pd.concat([data_1_1_ed, one_hoted_score],axis=1)
data_1_1_ed.drop(['信誉评级'],axis=1,inplace=True)
data_1_1_ed.head()

,是否违约,信誉评级_A,信誉评级_B,信誉评级_C,信誉评级_D
企业代号,,,,,
E1,0,1,0,0,0
E2,0,1,0,0,0
E3,0,0,0,1,0
E4,0,0,0,1,0
E5,0,0,1,0,0


但是独热编码认为所有的类别之间是没有大小关系的，但是信誉评级本身显然是有类似于优先级的次序的。所以我想也拿单纯的数字作为一个标签，这就是我上面为什么要copy一份data_1_1_ed

In [42]:
data_1_1_ed_2.head()

,信誉评级,是否违约
企业代号,,
E1,A,0
E2,A,0
E3,C,0
E4,C,0
E5,B,0


map() 会根据提供的函数对指定序列做映射。但是下面这行代码不能重复运行，反复执行map操作的话会因为找不到映射关系而把所有值变为NaN。

In [44]:
''' 
score_mapping = {'A':0,'B':1,'C':2,'D':3}
data_1_1_ed_2['信誉评级'] = data_1_1_ed_2['信誉评级'].map(score_mapping)
data_1_1_ed_2['信誉评级']
'''
# 在这里还是用Pandas提供的索引定位的办法,原因无他只是这是我当时用的233
data_1_1_ed_2.loc[data_1_1_ed_2['信誉评级']=='A','信誉评级'] = 0
data_1_1_ed_2.loc[data_1_1_ed_2['信誉评级']=='B','信誉评级'] = 1
data_1_1_ed_2.loc[data_1_1_ed_2['信誉评级']=='C','信誉评级'] = 2
data_1_1_ed_2.loc[data_1_1_ed_2['信誉评级']=='D','信誉评级'] = 3
data_1_1_ed_2.head()

,信誉评级,是否违约
企业代号,,
E1,0,0
E2,0,0
E3,2,0
E4,2,0
E5,1,0


将dataframe保存为csv文件，放入Cleaned_data文件夹下，指定编码方式防止乱码

In [45]:
data_1_1_ed.to_csv("Cleaned_data\label_one_hot.csv",encoding='gb18030')
data_1_1_ed_2.to_csv("Cleaned_data\label_without_one_hot.csv",encoding='gb18030')